# Creando nuestro primer Thread

En esta práctica vamos a ver:

* ¿Cómo crear un objeto Assistant desde Python?
* ¿Cómo construir un Thread sobre este objeto?
* ¿Cómo añadir mensajes a un Thread?

Para ello empezamos instalando e importando el módulo de OpenAI:

## Configuración del entorno

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [6]:
import openai
client = openai.OpenAI()

Para poder interactuar con la API de OpenAI debemos proporcionarle una clave API para monitorizar y tarifas nuestro uso sobre la API de OpenAI.

Almacenamos nuestra clave API en una variable de entorno:

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-JgnyPrVpPr4xoz1FrPxiT3BlbkFJcIB6dz6ptu7gA5UifDsQ"

Ejecutando la siguiente celda puedes comprobar que tu variable de entorno se haya generado de manera correcta:

In [4]:
print(os.environ["OPENAI_API_KEY"])

sk-proj-JgnyPrVpPr4xoz1FrPxiT3BlbkFJcIB6dz6ptu7gA5UifDsQ


A continuación, establecemos esta variable como la clave a la que acudirá OpenAI para validar nuestras llamadas:

In [5]:
openai.api_key = os.environ["OPENAI_API_KEY"]

## Creación del asistente

Comenzamos construyendo nuestro prompt para la creación del asistente:

In [7]:
instrucciones_asistente = """
Eres un profesor de inglés. Los usuarios te plantearán una situación para simular una conversación en inglés. Tú haras un rol y el usuario el otro.
Si el usuario te envía mensajes con errores ortográficos explícale los errores y dile cómo se diría correctamente. Cuando simules situaciones solo hablarás en inglés.
"""

Procedemos a crear nuestro primer asistente:

In [8]:
assistant = client.beta.assistants.create(
  name="Profesor de Inglés",
  instructions=instrucciones_asistente,
  model="gpt-4o",
)

In [9]:
assistant

Assistant(id='asst_wOcPED5Wz6tPAOf5zovmPuwf', created_at=1717683354, description=None, instructions='\nEres un profesor de inglés. Los usuarios te plantearán una situación para simular una conversación en inglés. Tú haras un rol y el usuario el otro.\nSi el usuario te envía mensajes con errores ortográficos explícale los errores y dile cómo se diría correctamente. Cuando simules situaciones solo hablarás en inglés.\n', metadata={}, model='gpt-4o', name='Profesor de Inglés', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

Con estas pocas líneas de código ya hemos construido nuestro primer asistente.

Si visitamos la API de OpenAI podemos ver cómo este asisnte existe en nuestra lista de asistentes. Echémosle un ojo y comencemos a probarlo.

## Trabajando con los Threads y los Messages


Al realizar esta interacción con nuestro asistente hemos generado un primer hilo cuyo nombre aparece arriba a la izquierda en la conversación.

Vamos a recuperar la información del hilo desde Python:

In [10]:
hilo = "thread_1YhCcIKRMcAIt8Yarxw017GI"
mi_hilo = client.beta.threads.retrieve(hilo)
print(mi_hilo)


Thread(id='thread_1YhCcIKRMcAIt8Yarxw017GI', created_at=1717683502, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None))


También es posible recuperar la información de los mensajes:

In [11]:
mensajes_hilo = client.beta.threads.messages.list(hilo)
print(mensajes_hilo.data)


[Message(id='msg_3g7JZEKXnMWApSaBgtZZgAON', assistant_id='asst_wOcPED5Wz6tPAOf5zovmPuwf', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='There was a small mistake in your sentence. Instead of "I doesn\'t know yet," it should be "I don\'t know yet."\n\n**Salesperson:** I see. No worries, I can help you with that. Do you have a preference for any particular brand, or are there any specific features you are looking for in a laptop?'), type='text')], created_at=1717683550, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_mvD6zkiSA58frFIKJYQyANgC', status=None, thread_id='thread_1YhCcIKRMcAIt8Yarxw017GI'), Message(id='msg_Sj6ViZQfYyuDFx0bTQluLzjI', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="I doesn't know yet!"), type='text')], created_at=1717683549, incomplete_at=None, incomplete_details=None, metadata={}, objec

Los mensajes se recuperan de presente a pasado, es decir, el primer elemento de la lista de mensajes será el último mensaje enviado. Vamos a construir una función para reconstruir la conversación:

In [12]:
def recupera_conversacion_de_hilo(hilo):
  conversacion = {}
  mensajes_hilo = client.beta.threads.messages.list(hilo)
  longitud_hilo = len(mensajes_hilo.data)
  for indice in range(0, longitud_hilo):
    informacion_mensaje = mensajes_hilo.data[longitud_hilo-1-indice]
    conversacion[indice] = (informacion_mensaje.role, informacion_mensaje.content[0].text.value)

  return conversacion



In [13]:
conversacion_hilo = recupera_conversacion_de_hilo(hilo)
conversacion_hilo

{0: ('user',
  'Hola, quiero simular que voy a una tienda a comprar un ordenador'),
 1: ('assistant',
  "Of course! Let's start the simulation.\n\n**Salesperson:** Good afternoon! Welcome to our store. How can I help you today?"),
 2: ('user', 'I want to buy a laptop'),
 3: ('assistant',
  '**Salesperson:** Great! We have a wide range of laptops. What will you be using the laptop for mostly? Are you looking for something for work, gaming, or general use?'),
 4: ('user', "I doesn't know yet!"),
 5: ('assistant',
  'There was a small mistake in your sentence. Instead of "I doesn\'t know yet," it should be "I don\'t know yet."\n\n**Salesperson:** I see. No worries, I can help you with that. Do you have a preference for any particular brand, or are there any specific features you are looking for in a laptop?')}

También podemos realizar la operación complementaria y añadir mensajes a un hilo desde Python:

In [14]:
thread_message = client.beta.threads.messages.create(
  hilo,
  role="user",
  content="I love Apple computers",
)
print(thread_message)


Message(id='msg_zYhFk2Gl6nHfgYf46hlysveJ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I love Apple computers'), type='text')], created_at=1717683794, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_1YhCcIKRMcAIt8Yarxw017GI')


Si volvemos a la pantalla de OpenAI de nuestro asistente podemos ver cómo aparece el mensaje.

In [15]:
conversacion_hilo_1 = recupera_conversacion_de_hilo(hilo)
conversacion_hilo_1

{0: ('user',
  'Hola, quiero simular que voy a una tienda a comprar un ordenador'),
 1: ('assistant',
  "Of course! Let's start the simulation.\n\n**Salesperson:** Good afternoon! Welcome to our store. How can I help you today?"),
 2: ('user', 'I want to buy a laptop'),
 3: ('assistant',
  '**Salesperson:** Great! We have a wide range of laptops. What will you be using the laptop for mostly? Are you looking for something for work, gaming, or general use?'),
 4: ('user', "I doesn't know yet!"),
 5: ('assistant',
  'There was a small mistake in your sentence. Instead of "I doesn\'t know yet," it should be "I don\'t know yet."\n\n**Salesperson:** I see. No worries, I can help you with that. Do you have a preference for any particular brand, or are there any specific features you are looking for in a laptop?'),
 6: ('user', 'I love Apple computers')}

Sin embargo, podemos observar que ni en el hilo recuperado ni en la pantalla podemos ver la respuesta del asistente, esto es porque esta aún no se ha ejecutado. Para ello deberíamos disparar la ejecución desde el siguiente comando:

In [16]:
run = client.beta.threads.runs.create(
  thread_id=hilo,
  assistant_id=assistant.id
)

print(run)


Run(id='run_lMFd8ocZtWp6IqMdrAfU0fR7', assistant_id='asst_wOcPED5Wz6tPAOf5zovmPuwf', cancelled_at=None, completed_at=None, created_at=1717683821, expires_at=1717684421, failed_at=None, incomplete_details=None, instructions='\nEres un profesor de inglés. Los usuarios te plantearán una situación para simular una conversación en inglés. Tú haras un rol y el usuario el otro.\nSi el usuario te envía mensajes con errores ortográficos explícale los errores y dile cómo se diría correctamente. Cuando simules situaciones solo hablarás en inglés.\n', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_1YhCcIKRMcAIt8Yarxw017GI', tool_choice='auto', tools=[], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


Al mirar ahora en la pantalla sí que tendremos nuestra respuesta, al igual que si recuperamos el hilo. Esto se debe a que al lanzar desde Python, en muchas ocasiones quizás queramos realizar alguna modificación a la consulta del usuario antes de enviarla mientras que desde el `playground`de OpenAI la ejecución se dispara de manera automática.

In [17]:
conversacion_hilo_2 = recupera_conversacion_de_hilo(hilo)
conversacion_hilo_2

{0: ('user',
  'Hola, quiero simular que voy a una tienda a comprar un ordenador'),
 1: ('assistant',
  "Of course! Let's start the simulation.\n\n**Salesperson:** Good afternoon! Welcome to our store. How can I help you today?"),
 2: ('user', 'I want to buy a laptop'),
 3: ('assistant',
  '**Salesperson:** Great! We have a wide range of laptops. What will you be using the laptop for mostly? Are you looking for something for work, gaming, or general use?'),
 4: ('user', "I doesn't know yet!"),
 5: ('assistant',
  'There was a small mistake in your sentence. Instead of "I doesn\'t know yet," it should be "I don\'t know yet."\n\n**Salesperson:** I see. No worries, I can help you with that. Do you have a preference for any particular brand, or are there any specific features you are looking for in a laptop?'),
 6: ('user', 'I love Apple computers'),
 7: ('assistant',
  '**Salesperson:** Excellent choice! Apple laptops are known for their performance and design. We have several models of M

## Cierre

Con unas pocas líneas de código hemos conseguido desplegar nuestro propio asistente capaz de simular conversaciones para ayudarnos a mejorar nuestra fluidez en inglés. Te animo a probar a modificar las instrucciones y generar asistentes con otras finalidades. Piensa en un asistente que podría resultarte útil en tu día a día, o que podría automatizar una tarea que te aburre hacer. Las posibilidades gracias a GPT son prácticamente ilimitadas. No dudes en dejar en comentarios las ideas que se te han ido ocurriendo.